In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import pandas as pd
import os
import json
import datetime
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [2]:
dataset_name = "STREAM"

# WARN
NOTE that timestamp is EXCATLY MATCHED WITH MODEL's timestamp (=> version/{timestamp})

In [3]:
# EXAMPLE: timestamp = "20210505-090740"
timestamp = ""

In [4]:
model = keras.models.load_model("version/{}".format(timestamp))

In [5]:
vocabulary_gpa = np.genfromtxt("static/vocabulary_gpa.csv", delimiter="\n", dtype=np.int64)
gpa_vocab_size = vocabulary_gpa.shape[0]
gpa_vocab_size

9770

In [6]:
vocabulary_rip = np.genfromtxt("static/vocabulary_rip.csv", delimiter="\n", dtype=np.uint64)
rip_vocab_size = vocabulary_rip.shape[0]
rip_vocab_size

5350

In [7]:
param_list = dict()

param_list["BATCH_SIZE"] = 64       # For AWS ml.p3.2xlarge instances
param_list["DILATIONS"] = [1, 2, 4, 8, 16, 32, 64, 128]
param_list["FILTER_WIDTH"] = 2                          # == kernel_size
param_list["SCALAR_INPUT"] = False
param_list["INITIAL_FILTER_WIDTH"] = 32     # Scalar Input
param_list["DILATION_CHANNELS"] = 32
param_list["RESIDUAL_CHANNELS"] = 24
param_list["SKIP_CHANNELS"] = 128
param_list["OUT_CHANNELS"] = gpa_vocab_size
param_list["USE_BIASES"] = False
param_list["BUFFER_SIZE"] = 200000
param_list["SHUFFLE_SEED"] = 102

if param_list["SCALAR_INPUT"]:
    param_list["RECEPTIVE_FIELD"] = (param_list["FILTER_WIDTH"] - 1) * sum(param_list["DILATIONS"]) + param_list["INITIAL_FILTER_WIDTH"]
else:
    param_list["RECEPTIVE_FIELD"] = (param_list["FILTER_WIDTH"] - 1) * sum(param_list["DILATIONS"]) + param_list["FILTER_WIDTH"]

param_list["MULTI_INPUT"] = True
param_list["INPUT_1_CHANNELS"] = param_list["OUT_CHANNELS"]
param_list["INPUT_2_CHANNELS"] = rip_vocab_size

In [8]:
model.summary()

Model: "wave_net_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
preprocessing/conv (Conv1D)  multiple                  725760    
_________________________________________________________________
residual_block_8 (ResidualBl multiple                  7936      
_________________________________________________________________
residual_block_9 (ResidualBl multiple                  7936      
_________________________________________________________________
residual_block_10 (ResidualB multiple                  7936      
_________________________________________________________________
residual_block_11 (ResidualB multiple                  7936      
_________________________________________________________________
residual_block_12 (ResidualB multiple                  7936      
_________________________________________________________________
residual_block_13 (ResidualB multiple                  7

In [9]:
test_set = pd.read_csv("data/{}_test_set.csv".format(dataset_name), dtype=np.int32)

In [10]:
x_test = tf.data.Dataset.from_tensor_slices(test_set[:-1].values).window(param_list["RECEPTIVE_FIELD"], 1, 1, True)
x_test = x_test.flat_map(lambda x: x.batch(param_list["RECEPTIVE_FIELD"])) 
x_test = x_test.batch(param_list["BATCH_SIZE"])

In [11]:
y_test = tf.data.Dataset.from_tensor_slices(test_set[param_list["RECEPTIVE_FIELD"]:]['gpa'].values).window(1, 1, 1, True)
y_test_slices = y_test.flat_map(lambda y: y.batch(1))
y_test = y_test_slices.map(lambda y: tf.one_hot(y, param_list["OUT_CHANNELS"], axis=-1))
y_test = y_test.batch(param_list["BATCH_SIZE"])

In [12]:
test_data = tf.data.Dataset.zip((x_test, y_test))

In [13]:
loss, acc = model.evaluate(test_data)

    362/Unknown - 17s 48ms/step - loss: 6516419.0000 - categorical_accuracy: 0.0000e+00

KeyboardInterrupt: 

In [13]:
#y_pred = tf.argmax(model.predict(x_test), axis=-1).numpy()     # Unable to predict whole test_set at a time
y_pred = []
for x in tqdm.tqdm(x_test):
    y_pred.extend(tf.argmax(model.predict(x), axis=-1).numpy())
y_pred = np.array(y_pred)

43710it [35:53, 20.30it/s]


In [14]:
y_true = np.array([yt for yt in y_test_slices.as_numpy_iterator()])

In [15]:
y_pred.shape, y_true.shape

((349673, 1), (349673, 1))

In [16]:
p, r, f = [], [], []
average_method = ["micro", "macro", "weighted"]

for method in average_method:
    precision = precision_score(np.ravel(y_true), np.ravel(y_pred), average=method)
    recall = recall_score(np.ravel(y_true), np.ravel(y_pred), average=method)
    f1 = f1_score(np.ravel(y_true), np.ravel(y_pred), average=method)
     
    p.append(precision)
    r.append(recall)
    f.append(f1)

In [17]:
report = pd.DataFrame(data=[p, r, f], columns=average_method, index=["precision", "recall", "f1"])
report

,micro,macro,weighted
precision,0.518862,0.014423,0.363667
recall,0.518862,0.005273,0.518862
f1,0.518862,0.005996,0.407918


In [18]:
accuracy = pd.DataFrame(data=[[loss, acc]], columns=["loss", "accuracy"])
accuracy

,loss,accuracy
0,3.664659,0.518862


In [19]:
report.to_csv("version/{}/report.csv".format(timestamp))
accuracy.to_csv("version/{}/accuracy.csv".format(timestamp), index=False)